In [ ]:
!kubectl label nodes apiserver-crd role=locust

# Start seldon-core

In [17]:
!helm install ../../helm-charts/seldon-core --name seldon-core --set grafana_prom_admin_password=password --set persistence.enabled=false --set cluster_manager.image.tag=0.2.16_kubeapi --set apife.image.tag=0.0.7_kubeapi

NAME:   seldon-core
LAST DEPLOYED: Tue Oct 31 18:09:12 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     6s
grafana-import-dashboards  5     6s
prometheus-rules           4     6s
prometheus-server-conf     1     6s

==> v1beta1/CustomResourceDefinition
NAME                                     AGE
mldeployments.machinelearning.seldon.io  6s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           6s

==> v1/Pod
NAME         READY  STATUS             RESTARTS  AGE
zookeeper-1  0/1    Pending            0         4s
zookeeper-2  0/1    Pending            0         4s
zookeeper-3  0/1    ContainerCreating  0         4s

==> v1/Pod(related)
grafana-prom-import-dashboards-l4r66      0/1  ContainerCreating  0  6s
alertmanager-deployment-779f586d4d-vvwfp  0/1  ContainerCreating  0  6s
seldon-apiserver-55c4558dcd-rcc6g         0/1  Container

# Integrating with Kubernetes API

In [30]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "MLDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mldeployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "cluster": {},
        "id": "1",
        "name": "test deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictor": {
            "annotations": {
                "project_name": "FX Market Prediction"
            },
            "enabled": true,
            "id": "0",
            "imagePullSecrets": [],
            "name": "FX Market Predictor",
            "predictiveUnits": [
                {
                    "children": [],
                    "cluster_resources": {
                        "cpu": "",
                        "gpu": "",
                        "id": "2",
                        "image": "seldonio/mean_cla

## Create ML Deployment

In [18]:
!kubectl apply -f resources/model.json

mldeployment "mldeployment-example" created


In [19]:
!kubectl get mldeployments

NAME                   AGE
mldeployment-example   5s


In [20]:
!kubectl describe mldeployments mldeployment-example 

Name:         mldeployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"MLDeployment","metadata":{"name":"mldeployment-example","namespace":"default","selfLink":"/a...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         MLDeployment
Metadata:
  Cluster Name:                   
  Creation Timestamp:             2017-10-31T18:11:26Z
  Deletion Grace Period Seconds:  0
  Finalizers:
  Generate Name:  
  Generation:     0
  Initializers:   <nil>
  Owner References:
  Resource Version:  4502
  Self Link:         /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/mldeployments/mldeployment-example
  UID:               e90be763-be66-11e7-97cc-08002751d93c
Spec:
  Annotations:
    Project _ Name:  FX Market Prediction
  Cluster:
  Id:              1
  Name:            test deployment
  Oauth _ Key:     oauth-key
  Oauth _ Secret:  oauth-secret


In [25]:
!kubectl get rs --selector=app=sd-1-p

NAME                DESIRED   CURRENT   READY     AGE
sd-1-c-54f5db79cd   1         1         1         2m
sd-1-p-c78c8f77f    1         1         1         21m


## Get predictions

In [7]:
%%bash
API_SERVER=35.195.94.148:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
#SERVER=0.0.0.0:8080
echo "$SERVER"
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

192.168.99.100:30032
{
  "meta": {
    "puid": "72v15lv0fj5leidrkucpa86ref",
    "routing": {
    }
  },
  "response": {
    "features": ["proba"],
    "ndarray": [[0.12823373759251927], [0.2856375705089441]]
  }
}

## Update deployment with canary

In [22]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "MLDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mldeployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "cluster": {},
        "id": "1",
        "name": "test deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictor": {
            "annotations": {
                "project_name": "FX Market Prediction"
            },
            "enabled": true,
            "id": "0",
            "imagePullSecrets": [],
            "name": "FX Market Predictor",
            "predictiveUnits": [
                {
                    "children": [],
                    "cluster_resources": {
                        "cpu": "",
                        "gpu": "",
                        "id": "2",
                        "image": "seldonio/mean_cla

In [23]:
!kubectl apply -f resources/model_with_canary.json

mldeployment "mldeployment-example" configured


In [24]:
!kubectl get rs --selector=app=sd-1-p

NAME                DESIRED   CURRENT   READY     AGE
sd-1-c-54f5db79cd   1         1         1         1m
sd-1-p-c78c8f77f    1         1         1         21m


In [26]:
%%bash
API_SERVER=35.189.249.200:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

{
  "meta": {
    "puid": "al91o30dcasj7mmaf8imnvt9mn",
    "routing": {
    }
  },
  "response": {
    "features": ["proba"],
    "ndarray": [[0.12823373759251927], [0.2856375705089441]]
  }
}

## Load test

In [ ]:
!helm install --set oauth.key=oauth-key --set oauth.secret=oauth-secret --name loadtest ../../helm-charts/seldon-core-loadtesting

# Tear down

In [27]:
!kubectl delete -f resources/model_with_canary.json

mldeployment "mldeployment-example" deleted


In [28]:
!helm delete loadtest --purge

Error: release: "loadtest" not found


In [29]:
!helm delete seldon-core --purge

release "seldon-core" deleted
